### Download Illumina sequence fastq files from Owl 

In [ ]:
# download using bash script
! bash ../scripts/download-quantseq.sh

### Install tag-seq scripts
I opted to install these scripts into my bioinformatics directory, where I house these types of programs. Create a {tagseq} variable

In [ ]:
! git clone https://github.com/z0on/tag-based_RNAseq.git /Applications/bioinformatics/tag-based_RNAseq/

In [13]:
# create path variable to blast directory 
tagseq = "/Applications/bioinformatics/tag-based_RNAseq/"

In [1]:
! ls ../data/illumina-raw/

CP-KS-LibL-L10-G_S59_L006_R1_001.fastq
CP-KS-LibL-L11-G_S65_L006_R1_001.fastq
CP-KS-LibL-L13-G_S60_L006_R1_001.fastq
CP-KS-LibL-L14-G_S63_L006_R1_001.fastq
CP-KS-LibL-L15-G_S68_L006_R1_001.fastq
CP-KS-LibL-L19-G_S67_L006_R1_001.fastq
CP-KS-LibL-L2-G_S34_L006_R1_001.fastq
CP-KS-LibL-L21-G_S58_L006_R1_001.fastq
CP-KS-LibL-L23-G_S61_L006_R1_001.fastq
CP-KS-LibL-L24-G_S62_L006_R1_001.fastq
CP-KS-LibL-L29-G_S36_L006_R1_001.fastq
CP-KS-LibL-L3-G_S64_L006_R1_001.fastq
CP-KS-LibL-L32-G_S35_L006_R1_001.fastq
CP-KS-LibL-L33-G_S66_L006_R1_001.fastq
CP-KS-LibL-L34-L_S25_L006_R1_001.fastq
CP-KS-LibL-L35-L_S31_L006_R1_001.fastq
CP-KS-LibL-L37-L_S32_L006_R1_001.fastq
CP-KS-LibL-L39-L_S29_L006_R1_001.fastq
CP-KS-LibL-L41-L_S27_L006_R1_001.fastq
CP-KS-LibL-L44-L_S30_L006_R1_001.fastq
CP-KS-LibL-L45-L_S28_L006_R1_001.fastq
CP-KS-LibL-L47-L_S33_L006_R1_001.fastq
CP-KS-LibL-L7-G_S57_L006_R1_001.fastq
CP-KS-LibL-SN1016-G_S26_L006_R1_001.fastq
clean
maps
quatseq-files.txt
tagSeq_trim.sh


### Adaptor trimming, deduplicating, and quality filtering:
Creating and launching the cleaning process for all files in the same time:

In [86]:
! ls {tagseq}

2bRAD_bowtie2_launch.pl        p5barcodes_order.xlsx
README.md                      picogreen.R
RNASeq_sample_prep_dec2017.pdf picogreen.csv
TagSeq_GSAF_Price.xlsx         rnaseq_clipper.pl
TagSeq_sample_prep_dec2017.pdf rnaseq_clipper0.pl
TagSeq_sample_prep_may2018.pdf rnaseq_clipper_fasta.pl
countreads.pl                  rnaseq_clipper_old.pl
dna.mixing.R                   samcount.pl
dupCount.R                     samcount_launch.pl
expression_compiler.pl         samcount_launch_bt2.pl
iRNAseq_bowtie2map.pl          samcount_v.0.1.pl
iRNAseq_shrimpmap_SAM.pl       selectFastaByHeader.pl
iRNAseq_trim_launch.pl         splitFastaByHeader.pl
iRNAseq_trim_launch0.pl        tag-seq_scripts_manual.pdf
iRNAseq_trim_launch_bgi.pl     tagSeq_processing_README.txt
illumina_mix_data.csv          tagseq_bowtie2map.pl
isogroup_namer.pl              tagseq_clipper.pl
launcher_creator.py            tagseq_oligo_order.xls
mix_illumina_qpcr.R            tagseq_trim_launch.pl
ngs_concat.pl          

In [90]:
# test fastq_quality_filter
! fastq_quality_filter -h

usage: fastq_quality_filter [-h] [-v] [-q N] [-p N] [-z] [-i INFILE] [-o OUTFILE]
Part of FASTX Toolkit 0.0.14 by A. Gordon (assafgordon@gmail.com)

   [-h]         = This helpful help screen.
   [-q N]       = Minimum quality score to keep.
   [-p N]       = Minimum percent of bases that must have [-q] quality.
   [-z]         = Compress output with GZIP.
   [-i INFILE]  = FASTA/Q input file. default is STDIN.
   [-o OUTFILE] = FASTA/Q output file. default is STDOUT.
   [-v]         = Verbose - report number of sequences.
                  If [-o] is specified,  report will be printed to STDOUT.
                  If [-o] is not specified (and output goes to STDOUT),
                  report will be printed to STDERR.



In [3]:
cd ../data/illumina-raw/

/Users/laura/Documents/roberts-lab/laura-quantseq/data/illumina-raw


In [78]:
# run perl script that creates list of executables for each fastq file 
! {tagseq}tagseq_trim_launch.pl '\.fastq$' > clean

In [4]:
! head -2 clean

tagseq_clipper.pl CP-KS-LibL-L21-G_S58_L006_R1_001.fastq | fastx_clipper -a AAAAAAAA -l 20 -Q33 | fastx_clipper -a AGATCGGAAG -l 20 -Q33 | fastq_quality_filter -Q33 -q 20 -p 90 >CP-KS-LibL-L21-G_S58_L006_R1_001.fastq.trim
tagseq_clipper.pl CP-KS-LibL-L23-G_S61_L006_R1_001.fastq | fastx_clipper -a AAAAAAAA -l 20 -Q33 | fastx_clipper -a AGATCGGAAG -l 20 -Q33 | fastq_quality_filter -Q33 -q 20 -p 90 >CP-KS-LibL-L23-G_S61_L006_R1_001.fastq.trim


### Based on repeated functions in the clean file, I wrote a script to loop through the files and execute all commands 

In [129]:
%%bash 

for file in *.fastq
do
#strip .fastq and directorys tructure from each file, then
# add suffice .fastq.trim to create output name for each file
results_file="$(basename -a $file).trim"

# run tagseq scripts on each file
/Applications/bioinformatics/tag-based_RNAseq/tagseq_clipper.pl $file | \
fastx_clipper -a AAAAAAAA -l 20 -Q33 | \
fastx_clipper -a AGATCGGAAG -l 20 -Q33 | \
fastq_quality_filter -Q33 -q 20 -p 90 >\
../tagseq_trim/$results_file 
done

CP-KS-LibL-L10-G_S59_L006_R1_001.fastq	total:6452865	goods:27350	dups:135439	noheader:6290063	N.in.header:13
CP-KS-LibL-L11-G_S65_L006_R1_001.fastq	total:5761774	goods:27135	dups:112294	noheader:5622331	N.in.header:14
CP-KS-LibL-L13-G_S60_L006_R1_001.fastq	total:4723646	goods:3787	dups:63418	noheader:4656437	N.in.header:4
CP-KS-LibL-L14-G_S63_L006_R1_001.fastq	total:5906432	goods:26663	dups:105949	noheader:5773809	N.in.header:11
CP-KS-LibL-L15-G_S68_L006_R1_001.fastq	total:9442881	goods:30660	dups:90323	noheader:9321885	N.in.header:13
CP-KS-LibL-L19-G_S67_L006_R1_001.fastq	total:6377544	goods:28528	dups:88628	noheader:6260379	N.in.header:9
CP-KS-LibL-L2-G_S34_L006_R1_001.fastq	total:5636566	goods:20425	dups:96398	noheader:5519737	N.in.header:6
CP-KS-LibL-L21-G_S58_L006_R1_001.fastq	total:4073358	goods:18890	dups:66504	noheader:3987951	N.in.header:13
CP-KS-LibL-L23-G_S61_L006_R1_001.fastq	total:6291394	goods:24916	dups:107619	noheader:6158846	N.in.header:13
CP-KS-LibL-L24-G_S62_L006_R1_

### Download O. lurida genome

In [88]:
! curl http://owl.fish.washington.edu/halfshell/genomic-databank/Olurida_v081.fa > ../references/Olurida_v081.fa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1090M  100 1090M    0     0  1960k      0  0:09:29  0:09:29 --:--:-- 2198k21k      0  0:08:46  0:00:17  0:08:29 2021k 0     0  2183k      0  0:08:31  0:01:11  0:07:20 2193k    0  2183k      0  0:08:31  0:01:14  0:07:17 2202k9  0:07:11 2212k1939k      0  0:09:35  0:01:45  0:07:50 1024k    0  1765k      0  0:10:32  0:02:00  0:08:32  497k   0     0  1752k      0  0:10:36  0:03:40  0:06:56 2190k    0  1748k      0  0:10:38  0:03:50  0:06:48 1105k  1855k      0  0:10:01  0:05:01  0:05:00 2215k  0     0  1861k      0  0:09:59  0:05:07  0:04:52 2191k   0  1881k      0  0:09:53  0:05:25  0:04:28 2222k931k      0  0:09:38  0:07:31  0:02:07 2169k  1929k      0  0:09:38  0:08:03  0:01:35 2179k 0  1933k      0  0:09:37  0:08:11  0:01:26 2200k  0  0:09:35  0:08:42  0:00:53 2212k


In [89]:
# MD5 should = 3ac56372bd62038f264d27eef0883bd3
! md5 ../references/Olurida_v081.fa

MD5 (../references/Olurida_v081.fa) = 3ac56372bd62038f264d27eef0883bd3


In [30]:
! mkdir bowtie/

In [34]:
cd bowtie/

/Users/laura/Documents/roberts-lab/laura-quantseq/results/bowtie


In [37]:
%%bash
### creating bowtie2 index for Oly genome v081:

bowtie2-build \
../../references/Olurida_v081.fa \
Olurida_v081.fa

Settings:
  Output files: "Olurida_v081.fa.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ../../references/Olurida_v081.fa
Reading reference sizes
  Time reading reference sizes: 00:00:12
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:07
bmax according to bmaxDivN setting: 269313774
Using parameters --bmax 201985331 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 201985331 --dcv 1024
Constructing suffix-

Building a SMALL index


In [39]:
! ls

Olurida_v081.fa.1.bt2     Olurida_v081.fa.3.bt2     Olurida_v081.fa.rev.1.bt2
Olurida_v081.fa.2.bt2     Olurida_v081.fa.4.bt2     Olurida_v081.fa.rev.2.bt2


In [41]:
cd ../../data/tagseq_trim/

/Users/laura/Documents/roberts-lab/laura-quantseq/data/tagseq_trim


In [104]:
! {tagseq}tagseq_bowtie2map.pl "trim$" ../../results/bowtie/Olurida_v081.fa > maps

In [105]:
! head -2 maps

bowtie2 --local -x ../../results/bowtie/Olurida_v081.fa -U CP-KS-LibL-L37-L_S32_L006_R1_001.fastq.trim -S CP-KS-LibL-L37-L_S32_L006_R1_001.fastq.trim.sam --no-hd --no-sq --no-unal -k 5
bowtie2 --local -x ../../results/bowtie/Olurida_v081.fa -U CP-KS-LibL-L3-G_S64_L006_R1_001.fastq.trim -S CP-KS-LibL-L3-G_S64_L006_R1_001.fastq.trim.sam --no-hd --no-sq --no-unal -k 5


###  Use a loop to execute the commands written in 'maps'; redirect screen output/error in bowtieout.txt

In [106]:
cd ../../data/tagseq_trim/

/Users/laura/Documents/roberts-lab/laura-quantseq/data/tagseq_trim


--no-unal  = suppress SAM records for unaligned reads  
--no-sq    = suppress @SQ header lines  
--no-hd    = suppress SAM header lines (starting with @)  
-k 5       = report up to <5> aligns per read  

In [110]:
%%bash 

for file in *.trim
do
#strip .fastq and directorys tructure from each file, then
# add suffice .fastq.trim to create output name for each file
results_file="$(basename -a $file).sam"

# run tagseq scripts on each file
bowtie2 --local -x \
../../results/bowtie/Olurida_v081.fa \
-U $file \
-S $results_file \ -k 5; \
done >> ../../results/bowtie/bowtieout.txt 2>&1

In [111]:
! cat ../../results/bowtie/bowtieout.txt

21874 reads; of these:
  21874 (100.00%) were unpaired; of these:
    6373 (29.14%) aligned 0 times
    11759 (53.76%) aligned exactly 1 time
    3742 (17.11%) aligned >1 times
70.86% overall alignment rate
21768 reads; of these:
  21768 (100.00%) were unpaired; of these:
    6229 (28.62%) aligned 0 times
    11759 (54.02%) aligned exactly 1 time
    3780 (17.36%) aligned >1 times
71.38% overall alignment rate
2649 reads; of these:
  2649 (100.00%) were unpaired; of these:
    2067 (78.03%) aligned 0 times
    352 (13.29%) aligned exactly 1 time
    230 (8.68%) aligned >1 times
21.97% overall alignment rate
21489 reads; of these:
  21489 (100.00%) were unpaired; of these:
    6142 (28.58%) aligned 0 times
    11731 (54.59%) aligned exactly 1 time
    3616 (16.83%) aligned >1 times
71.42% overall alignment rate
23903 reads; of these:
  23903 (100.00%) were unpaired; of these:
    6156 (25.75%) aligned 0 times
    13075 (54.70%) aligned exactly 1 time
    4672 (19.55%) aligned >1 times
7

In [112]:
# alignment rates:
! grep "overall alignment rate"  ../../results/bowtie/bowtieout.txt

70.86% overall alignment rate
71.38% overall alignment rate
21.97% overall alignment rate
71.42% overall alignment rate
74.25% overall alignment rate
74.05% overall alignment rate
69.84% overall alignment rate
71.05% overall alignment rate
69.79% overall alignment rate
65.44% overall alignment rate
73.34% overall alignment rate
41.75% overall alignment rate
73.35% overall alignment rate
73.44% overall alignment rate
63.80% overall alignment rate
73.59% overall alignment rate
57.34% overall alignment rate
71.77% overall alignment rate
74.38% overall alignment rate
74.85% overall alignment rate
62.80% overall alignment rate
68.49% overall alignment rate
73.56% overall alignment rate
71.53% overall alignment rate


## Generating read-counts-per gene 

In [113]:
ls

CP-KS-LibL-L10-G_S59_L006_R1_001.fastq.trim
CP-KS-LibL-L10-G_S59_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L11-G_S65_L006_R1_001.fastq.trim
CP-KS-LibL-L11-G_S65_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L13-G_S60_L006_R1_001.fastq.trim
CP-KS-LibL-L13-G_S60_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L14-G_S63_L006_R1_001.fastq.trim
CP-KS-LibL-L14-G_S63_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L15-G_S68_L006_R1_001.fastq.trim
CP-KS-LibL-L15-G_S68_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L19-G_S67_L006_R1_001.fastq.trim
CP-KS-LibL-L19-G_S67_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L2-G_S34_L006_R1_001.fastq.trim
CP-KS-LibL-L2-G_S34_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L21-G_S58_L006_R1_001.fastq.trim
CP-KS-LibL-L21-G_S58_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L23-G_S61_L006_R1_001.fastq.trim
CP-KS-LibL-L23-G_S61_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L24-G_S62_L006_R1_001.fastq.trim
CP-KS-LibL-L24-G_S62_L006_R1_001.fastq.trim.sam
CP-KS-LibL-L29-G_S36_L006_R1_001.fastq.trim
CP-KS-LibL-L29-G_S36_L006_R1_001.fastq

In [82]:
! grep ">" ../../references/Olurida_v081.fa | head

>Contig0
>Contig1
>Contig2
>Contig3
>Contig4
>Contig5
>Contig6
>Contig7
>Contig8
>Contig9


In [63]:
! samtools view --help

samtools view: unrecognised option '--help'

Usage: samtools view [options] <in.bam>|<in.sam>|<in.cram> [region ...]

Options:
  -b       output BAM
  -C       output CRAM (requires -T)
  -1       use fast BAM compression (implies -b)
  -u       uncompressed BAM output (implies -b)
  -h       include header in SAM output
  -H       print SAM header only (no alignments)
  -c       print only the count of matching records
  -o FILE  output file name [stdout]
  -U FILE  output reads not selected by filters to FILE [null]
  -t FILE  FILE listing reference names and lengths (see long help) [null]
  -L FILE  only include reads overlapping this BED FILE [null]
  -r STR   only include reads in read group STR [null]
  -R FILE  only include reads with read group listed in FILE [null]
  -q INT   only include reads with mapping quality >= INT [0]
  -l STR   only include reads in library STR [null]
  -m INT   only include reads with number of CIGAR operations consuming
           query sequence >= 

In [114]:
%%bash
#convert sam to bam

for file in *.sam
do
results_file="$(basename -a $file)_sorted.bam"
samtools view -b $file | samtools sort -o $results_file
done

In [115]:
%%bash
# create .bam indexes
for file in *.bam
do
samtools index $file
done

In [118]:
ls *.bam

CP-KS-LibL-L10-G_S59_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L11-G_S65_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L13-G_S60_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L14-G_S63_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L15-G_S68_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L19-G_S67_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L2-G_S34_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L21-G_S58_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L23-G_S61_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L24-G_S62_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L29-G_S36_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L3-G_S64_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L32-G_S35_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L33-G_S66_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L34-L_S25_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L35-L_S31_L006_R1_001.fastq.trim.sam_sorted.bam
CP-KS-LibL-L37-L_S32_L006_R1_001.fastq.trim.sam_sorted.bam

In [121]:
# total mapped and paired reads 
! samtools view -F 6 \
CP-KS-LibL-L7-G_S57_L006_R1_001.fastq.trim.sam_sorted.bam | \
wc -l

   27324


In [122]:
# weird, this is supposed to be the unmapped read count .. same as above
! samtools view -F 4 \
CP-KS-LibL-L7-G_S57_L006_R1_001.fastq.trim.sam_sorted.bam | \
wc -l

   27324
